# Multi-LLM Collaborative Debate System - Demo Playground

Interactive notebook for running single debate sessions and observing agent reasoning in real-time.

## Features:
- Run debates on individual problems
- Observe each agent's reasoning process
- See peer reviews and refinements
- Watch the judge make the final decision

In [ ]:
import json
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path(".").resolve().parent))

from dotenv import load_dotenv
from openai import OpenAI

from src.agents import PERSONAS
from src.orchestrator import (
    get_role_preference,
    generate_solution,
    generate_critique,
    refine_solution,
    judge_verdict,
    grade_answer,
)

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI()
print("OpenAI client initialized successfully!")

## 1. Load Problem Set

In [ ]:
PROBLEMS_PATH = Path("../data/problems.json")

with open(PROBLEMS_PATH, "r") as f:
    problems = json.load(f)

print(f"Loaded {len(problems)} problems")
print("\nAvailable problems:")
for p in problems[:10]:  # Show first 10
    print(f"  [{p['id']}] {p['category']}/{p['difficulty']}: {p['question'][:60]}...")

## 2. Select a Problem

Choose a problem ID or define a custom problem.

In [ ]:
# Option 1: Select by ID
PROBLEM_ID = 1  # Change this to test different problems

problem = next((p for p in problems if p["id"] == PROBLEM_ID), None)

# Option 2: Define a custom problem (uncomment to use)
# problem = {
#     "id": 999,
#     "category": "Custom",
#     "difficulty": "Medium",
#     "question": "What is 2 + 2?",
#     "ground_truth": "4",
# }

if problem:
    print(f"Selected Problem {problem['id']}")
    print(f"Category: {problem['category']}")
    print(f"Difficulty: {problem['difficulty']}")
    print(f"\nQuestion:\n{problem['question']}")
    print(f"\nGround Truth: {problem['ground_truth']}")
else:
    print(f"Problem ID {PROBLEM_ID} not found!")

## 3. Helper Functions for Pretty Printing

In [ ]:
def print_header(title: str) -> None:
    """Print a formatted header."""
    print("\n" + "=" * 80)
    print(f" {title}")
    print("=" * 80)


def print_subheader(title: str) -> None:
    """Print a formatted subheader."""
    print(f"\n--- {title} ---")


def print_agent_info(agent_id: str) -> None:
    """Print agent persona information."""
    print(f"\nAgent {agent_id} Persona: {PERSONAS[agent_id][:100]}...")

## 4. Stage 0: Role Assignment

Each agent decides if they prefer to be a Solver or Judge.

In [ ]:
print_header("STAGE 0: ROLE ASSIGNMENT")

question = problem["question"]
agent_ids = list(PERSONAS.keys())
preferences = []

for agent_id in agent_ids:
    print_subheader(f"Agent {agent_id}")
    print_agent_info(agent_id)
    
    pref = get_role_preference(client, agent_id, question)
    preferences.append(pref)
    
    print(f"\nPreferred Role: {pref.role_priority}")
    print(f"Confidence: {pref.confidence:.2f}")
    print(f"Reasoning: {pref.reasoning[:200]}...")

In [ ]:
# Assign roles based on preferences
import random

judge_candidates = [p for p in preferences if p.role_priority == "Judge"]

if judge_candidates:
    weights = [p.confidence for p in judge_candidates]
    judge_pref = random.choices(judge_candidates, weights=weights, k=1)[0]
    judge_id = judge_pref.agent_id
else:
    judge_pref = random.choice(preferences)
    judge_id = judge_pref.agent_id

solver_ids = [p.agent_id for p in preferences if p.agent_id != judge_id]

print_subheader("ROLE ASSIGNMENT RESULT")
print(f"Judge: Agent {judge_id}")
print(f"Solvers: Agents {', '.join(solver_ids)}")

## 5. Stage 1: Independent Solutions

Each solver generates their solution independently.

In [ ]:
print_header("STAGE 1: INDEPENDENT SOLUTIONS")

initial_solutions = {}

for solver_id in solver_ids:
    print_subheader(f"Solver {solver_id}")
    print_agent_info(solver_id)
    
    solution = generate_solution(client, solver_id, question)
    initial_solutions[solver_id] = solution
    
    print(f"\nSolution:")
    print(f"{solution.solution_text[:500]}..." if len(solution.solution_text) > 500 else solution.solution_text)
    print(f"\nFINAL ANSWER: {solution.final_answer}")

## 6. Stage 2: Peer Review (Round Robin)

Each solver critiques the other two solvers' work.

In [ ]:
print_header("STAGE 2: PEER REVIEW")

all_reviews = {sid: [] for sid in solver_ids}

for reviewer_id in solver_ids:
    for target_id in solver_ids:
        if reviewer_id != target_id:
            print_subheader(f"Reviewer {reviewer_id} -> Target {target_id}")
            
            review = generate_critique(
                client,
                reviewer_id,
                target_id,
                question,
                initial_solutions[target_id],
            )
            all_reviews[target_id].append(review)
            
            print(f"\nScore: {review.score}/10")
            print(f"Strengths: {', '.join(review.strengths[:3])}")
            print(f"Weaknesses: {', '.join(review.weaknesses[:3])}")
            if review.errors:
                print(f"Errors Found: {len(review.errors)}")
                for err in review.errors[:2]:
                    print(f"  - [{err.severity}] {err.location}: {err.description[:100]}")

## 7. Stage 3: Refinement

Solvers improve their solutions based on peer feedback.

In [ ]:
print_header("STAGE 3: REFINEMENT")

refined_solutions = {}

for solver_id in solver_ids:
    print_subheader(f"Solver {solver_id} Refining")
    
    refined = refine_solution(
        client,
        solver_id,
        question,
        initial_solutions[solver_id],
        all_reviews[solver_id],
    )
    refined_solutions[solver_id] = refined
    
    print(f"\nChanges Made: {refined.changes_made[:300]}..." if len(refined.changes_made) > 300 else f"\nChanges Made: {refined.changes_made}")
    print(f"\nOriginal Answer: {initial_solutions[solver_id].final_answer}")
    print(f"Refined Answer:  {refined.final_answer}")
    
    # Check if answer changed
    if initial_solutions[solver_id].final_answer != refined.final_answer:
        print(">>> ANSWER CHANGED!")

## 8. Stage 4: Judge Verdict

The judge evaluates all solutions and selects the winner.

In [ ]:
print_header("STAGE 4: JUDGE VERDICT")

print(f"\nJudge: Agent {judge_id}")
print_agent_info(judge_id)

verdict = judge_verdict(
    client,
    judge_id,
    question,
    solver_ids,
    initial_solutions,
    all_reviews,
    refined_solutions,
)

print(f"\nWINNER: Solver {verdict.best_solver_id}")
print(f"\nRationale:")
print(verdict.rationale)
print(f"\nFINAL ANSWER TO USER: {verdict.final_answer_to_user}")

## 9. Evaluation

Compare the final answer against the ground truth.

In [ ]:
print_header("EVALUATION")

ground_truth = problem["ground_truth"]

evaluation = grade_answer(
    client,
    question,
    ground_truth,
    verdict.final_answer_to_user,
)

print(f"\nGround Truth: {ground_truth}")
print(f"System Answer: {verdict.final_answer_to_user}")
print(f"\nCorrect: {'YES' if evaluation.is_correct else 'NO'}")
print(f"\nEvaluation Reasoning:")
print(evaluation.reasoning)

## 10. Summary

In [ ]:
print_header("DEBATE SUMMARY")

print(f"""
Problem ID: {problem['id']}
Category: {problem['category']}
Difficulty: {problem['difficulty']}

Roles:
  - Judge: Agent {judge_id}
  - Solvers: {', '.join(solver_ids)}

Initial Answers:
""")
for sid in solver_ids:
    print(f"  - Solver {sid}: {initial_solutions[sid].final_answer}")

print(f"""
Refined Answers:
""")
for sid in solver_ids:
    changed = "(changed)" if initial_solutions[sid].final_answer != refined_solutions[sid].final_answer else ""
    print(f"  - Solver {sid}: {refined_solutions[sid].final_answer} {changed}")

print(f"""
Judge Selected: Solver {verdict.best_solver_id}
Final Answer: {verdict.final_answer_to_user}

Ground Truth: {ground_truth}
Result: {'CORRECT' if evaluation.is_correct else 'INCORRECT'}
""")

---

## Quick Run: Full Debate Pipeline

Use this cell to run a complete debate in one go.

In [ ]:
def run_full_debate(problem_id: int) -> dict:
    """Run a complete debate for a given problem ID."""
    problem = next((p for p in problems if p["id"] == problem_id), None)
    if not problem:
        raise ValueError(f"Problem ID {problem_id} not found")
    
    question = problem["question"]
    print(f"Running debate for Problem {problem_id}: {question[:80]}...")
    
    # Stage 0: Role assignment
    preferences = [get_role_preference(client, aid, question) for aid in PERSONAS.keys()]
    judge_candidates = [p for p in preferences if p.role_priority == "Judge"]
    if judge_candidates:
        judge_id = random.choices(judge_candidates, weights=[p.confidence for p in judge_candidates], k=1)[0].agent_id
    else:
        judge_id = random.choice(preferences).agent_id
    solver_ids = [p.agent_id for p in preferences if p.agent_id != judge_id]
    print(f"  Roles: Judge={judge_id}, Solvers={solver_ids}")
    
    # Stage 1: Solutions
    initial_solutions = {sid: generate_solution(client, sid, question) for sid in solver_ids}
    print(f"  Initial answers: {[initial_solutions[s].final_answer[:30] for s in solver_ids]}")
    
    # Stage 2: Reviews
    all_reviews = {sid: [] for sid in solver_ids}
    for reviewer in solver_ids:
        for target in solver_ids:
            if reviewer != target:
                all_reviews[target].append(
                    generate_critique(client, reviewer, target, question, initial_solutions[target])
                )
    print(f"  Peer reviews completed")
    
    # Stage 3: Refinement
    refined_solutions = {
        sid: refine_solution(client, sid, question, initial_solutions[sid], all_reviews[sid])
        for sid in solver_ids
    }
    print(f"  Refined answers: {[refined_solutions[s].final_answer[:30] for s in solver_ids]}")
    
    # Stage 4: Verdict
    verdict = judge_verdict(client, judge_id, question, solver_ids, initial_solutions, all_reviews, refined_solutions)
    print(f"  Judge selected: Solver {verdict.best_solver_id}")
    
    # Evaluation
    evaluation = grade_answer(client, question, problem["ground_truth"], verdict.final_answer_to_user)
    print(f"  Final answer: {verdict.final_answer_to_user}")
    print(f"  Ground truth: {problem['ground_truth']}")
    print(f"  Result: {'CORRECT' if evaluation.is_correct else 'INCORRECT'}")
    
    return {
        "problem": problem,
        "verdict": verdict,
        "evaluation": evaluation,
    }


# Example: Run on problem 7 (Monty Hall)
# result = run_full_debate(7)